## 手作業によるコンテンツベースのフィルタリング

このラボでは、低レベルのTensorflow操作を使用してコンテンツベースのフィルターを実装する方法を説明します。
ここでのコードは、レコメンデーションエンジンのモジュール2：コンテンツベースのフィルタリングで説明されている手法に従います。


In [1]:
!pip install tensorflow==2.5

     |████████████████████████████████| 454.3 MB 10 kB/s              ��███████████████████▊  | 422.7 MB 89.7 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 60.8 MB/s            
     |████████████████████████████████| 1.2 MB 50.6 MB/s            
     |████████████████████████████████| 4.0 MB 56.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 152 kB 56.4 MB/s            
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=76320 sha256=b1fb0050866e6c4a7829e19775d9d776b7a8da5047f1ac0ea098a4b582421396
  Stored in directory: /home/jupyter/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing inst

この変更が行われたことを確認するために、必ずカーネルを再起動してください。

In [1]:
import numpy as np
import tensorflow as tf

print(tf.__version__)

2021-11-07 15:02:53.435996: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


2.5.0


まず、ユーザー、映画、機能のリストを作成します。ユーザーと映画はデータベース内の要素を表していますが、コンテンツベースのフィルタリング方法の場合、映画の機能はおそらく手作業で設計されており、ドメイン知識に依存して最適な埋め込みスペースを提供します。ここでは、アクション、SF、コメディ、漫画、ドラマのカテゴリを使用して、映画（したがってユーザー）を説明します。

この例では、データベースが以下にリストされている4人のユーザーと6本の映画で構成されていると想定します。

In [2]:
users = ['Ryan', 'Danielle',  'Vijay', 'Chris']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memento']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2

### ユーザー、映画のレイティング、機能を初期化する

ユーザーの映画のレーティングとk-hotでエンコードされた映画の機能マトリックスを入力する必要があります。 users_moviesマトリックスの各行は、各映画に対する1人のユーザーの評価（1から10）を表します。ゼロは、ユーザーがその映画を見ていない/評価していないことを示します。 movies_featsマトリックスには、指定された各映画の機能が含まれています。各行は6つの映画のいずれかを表し、列は5つのカテゴリを表します。 1は、映画が特定のジャンル/カテゴリに適合することを示します。

In [3]:
# 各行は、さまざまな映画に対するユーザーの評価を表します
users_movies = tf.constant([
                [4,  6,  8,  0, 0, 0],
                [0,  0, 10,  0, 8, 3],
                [0,  6,  0,  0, 3, 7],
                [10, 9,  0,  5, 0, 2]],dtype=tf.float32)

# ワンホットエンコードされた映画の機能
# 例えば列は表すことができます ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]],dtype=tf.float32)

2021-11-07 15:05:45.128856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-07 15:05:45.128903: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-07 15:05:45.128940: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-6-20211107-235606): /proc/driver/nvidia/version does not exist
2021-11-07 15:05:45.132061: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler fla

### ユーザーの特徴量マトリックスの計算

ユーザー機能マトリックスを計算します。つまり、5次元の特徴空間への各ユーザーの埋め込みを含む行列です。

In [4]:
users_feats = tf.matmul(users_movies,movies_feats)
users_feats

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[10., 10.,  8.,  8.,  4.],
       [ 3.,  0., 10., 10., 11.],
       [13.,  6.,  0.,  0., 10.],
       [26., 19.,  5.,  5., 12.]], dtype=float32)>

次に、各ユーザーの特徴ベクトルを正規化して合計を1にします。正規化は厳密には必要ではありませんが、評価の大きさがユーザー間で比較できるようにします。

In [5]:
users_feats = users_feats/tf.reduce_sum(users_feats,axis=1,keepdims=True)
users_feats

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

#### 各ユーザーのランキング機能の関連性

上で計算されたusers_featsを使用して、各ユーザーの各映画カテゴリの相対的な重要性を表すことができます。

In [6]:
top_users_features = tf.nn.top_k(users_feats, num_feats)[1]
top_users_features

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)>

In [7]:
for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i],feature_names))

Ryan: ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
Danielle: ['Drama', 'Comedy', 'Cartoon', 'Action', 'Sci-Fi']
Vijay: ['Action', 'Drama', 'Sci-Fi', 'Comedy', 'Cartoon']
Chris: ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon']


### 映画のおすすめを決定する。

ここで、上記で計算した `users_feats`テンソルを使用して、各ユーザーの映画の評価と推奨事項を決定します。

各映画の予測評価を計算するために、ユーザーの特徴ベクトルと対応する映画の特徴ベクトルの間の類似度を計算します。

類似性の尺度として内積を使用します。本質的に、これは各ユーザーの加重映画の平均です。

In [8]:
users_ratings = tf.matmul(users_feats,tf.transpose(movies_feats))
users_ratings

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.5882353 , 0.67647064, 0.32352942,
        0.4117647 ],
       [1.        , 0.6551724 , 0.        , 0.44827586, 0.3448276 ,
        0.79310346],
       [0.8507463 , 0.6716418 , 0.14925373, 0.53731346, 0.17910448,
        0.5671642 ]], dtype=float32)>

上記の計算により、データベース内の各ユーザーと各映画の間の類似度が検出されます。新しい映画の評価のみに焦点を合わせるために、all_users_ratingsマトリックスにマスクを適用します。

ユーザーがすでに映画を評価している場合、その評価は無視されます。このように、私たちは以前に見られなかった/評価されていない映画の評価にのみ焦点を合わせます。

In [9]:
users_ratings_new = tf.where(tf.equal(users_movies, tf.zeros_like(users_movies)),
                                  users_ratings,
                                  tf.zeros_like(tf.cast(users_movies, tf.float32)))
users_ratings_new

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.        , 0.67647064, 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.44827586, 0.        ,
        0.        ],
       [0.        , 0.        , 0.14925373, 0.        , 0.17910448,
        0.        ]], dtype=float32)>

最後に、各ユーザーの評価の高い映画の上位2つを取得して印刷しましょう

In [10]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommendations)[1]
top_movies

<tf.Tensor: shape=(4, 2), dtype=int32, numpy=
array([[3, 5],
       [3, 0],
       [0, 3],
       [4, 2]], dtype=int32)>

In [11]:
for i in range(num_users):
    movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i],movie_names))

Ryan: ['The Incredibles', 'Memento']
Danielle: ['The Incredibles', 'Star Wars']
Vijay: ['Star Wars', 'The Incredibles']
Chris: ['Bleu', 'Shrek']
